<a href="https://colab.research.google.com/github/dmistry1/KNN-Classifier/blob/main/Knn_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

# Problem 1:


---



In [2]:
# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Function to train and evaluate KNN classifier
def train_and_evaluate(X_train, X_test, y_train, y_test, n_neighbors):
    start_time = time.time()
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    end_time = time.time()
    runtime = end_time - start_time
    return accuracy, runtime

# Define different values of N
# N_values = [2000, 4000]
N_values = [3000, 5000]
# N_values = [1500, 3000]

# Empty lists to store results
accuracies = []
runtimes = []

# Iterate over N_values
for N in N_values:
    # Subset the training set
    X_train_subset = []
    y_train_subset = []
    for i in range(10):  # Fashion MNIST has 10 classes
        X_class = X_train[y_train == i][:N//10]
        y_class = y_train[y_train == i][:N//10]
        X_train_subset.extend(X_class)
        y_train_subset.extend(y_class)
    X_train_subset = np.array(X_train_subset)
    y_train_subset = np.array(y_train_subset)

    # Train and evaluate KNN
    accuracy, runtime = train_and_evaluate(X_train_subset.reshape(-1, 784), X_test.reshape(-1, 784), y_train_subset, y_test, n_neighbors=5)

    # Append results
    accuracies.append(accuracy)
    runtimes.append(runtime)

# Print results
for N, accuracy, runtime in zip(N_values, accuracies, runtimes):
    print(f"N = {N}, Accuracy = {accuracy:.4f}, Runtime = {runtime:.2f} seconds")

4422102/4422102 [==============================] - 0s 0us/step
N = 3000, Accuracy = 0.7843, Runtime = 2.87 seconds
N = 5000, Accuracy = 0.8020, Runtime = 5.21 seconds


In the code block above, I have tests these followig values. Here are my observations.

**N1=2000, N2=4000**
```
N = 2000, Accuracy = 0.7765, Runtime = 2.05 seconds
N = 4000, Accuracy = 0.7946, Runtime = 4.48 seconds
```

These values is a good balance between efficiency and model performance.

N1 = 2000 for each class would provide a reasonably sized training set for each class without overwhelming computational resources.

N2 = 4000 for each class could provide a larger dataset for better model performance without excessively increasing runtime.

**N1=3000, N2=5000**
```
N = 3000, Accuracy = 0.7843, Runtime = 3.94 seconds
N = 5000, Accuracy = 0.8020, Runtime = 4.13 seconds
```

These values lean towards better model performance at the cost of increased runtime.

N1 = 3000 for each class provides more data for training without a significant increase in runtime.

N2 = 5000 for each class offers even more data for potentially better model generalization.

**N1=1500, N2=3000**
```
N = 1500, Accuracy = 0.7669, Runtime = 1.76 seconds
N = 3000, Accuracy = 0.7843, Runtime = 2.54 seconds
```

These values prioritize computational efficiency while still providing a reasonable amount of data for training.

N1 = 1500 for each class reduces computational overhead while still allowing the model to learn from a diverse set of examples.

N2 = 3000 for each class balances computational efficiency with improved model performance compared to smaller training sets.




# Problem 2a:


---



The algorithm that we will do is the following:
- For each class in Fashion MIST:
 - Get the indices of samples belonging to that class.
 - Randomly shuffle the indices.
 - Select the first N1 indices for the training set.
 - Select the remaining samples (7000 - N1) for the testing set.
- Repeat the process for all classes.

In [3]:
# Define N1
N1 = 4000

# Initialize lists to store training and testing data
X_train_split = []
X_test_split = []
y_train_split = []
y_test_split = []

# Iterate over each class
for class_label in range(10):  # Fashion MNIST has 10 classes
    # Get indices of samples belonging to the current class
    class_indices = np.where(y_train == class_label)[0]

    # Shuffle indices
    np.random.shuffle(class_indices)

    # Select first N1 indices for training set
    train_indices = class_indices[:N1]
    X_train_split.extend(X_train[train_indices])
    y_train_split.extend(y_train[train_indices])

    # Select remaining samples for testing set
    test_indices = class_indices[N1:7000]  # 7000 - N1
    X_test_split.extend(X_train[test_indices])
    y_test_split.extend(y_train[test_indices])

# Converting the lists to numpy arrays
X_train_split = np.array(X_train_split)
X_test_split = np.array(X_test_split)
y_train_split = np.array(y_train_split)
y_test_split = np.array(y_test_split)

# Print shapes of resulting arrays
print("Shape of X_train_split:", X_train_split.shape)
print("Shape of X_test_split:", X_test_split.shape)
print("Shape of y_train_split:", y_train_split.shape)
print("Shape of y_test_split:", y_test_split.shape)

Shape of X_train_split: (40000, 28, 28)
Shape of X_test_split: (20000, 28, 28)
Shape of y_train_split: (40000,)
Shape of y_test_split: (20000,)


# Problem 2b:


---



In [4]:
# Flatten the images
X_train_flat = X_train_split.reshape(-1, 784)
X_test_flat = X_test_split.reshape(-1, 784)

# Define fixed k value
k = 20

# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_flat, y_train_split)

# Make predictions on the test set
y_pred = knn.predict(X_test_flat)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)
print(f"Global success rate with N1 = {N1}: {accuracy:.4f}")

Global success rate with N1 = 4000: 0.8404


# Problem 2c:


---



In [5]:
# Define values of N1, N2, and N3
N_values = [3000, 4000, 5000]

# Define fixed k value
k = 20

# Initialize empty lists to store accuracies
accuracies_N = []

# Iterate over each value of N1, N2, and N3
for N in N_values:
    # Initialize lists to store training and testing data
    X_train_split = []
    X_test_split = []
    y_train_split = []
    y_test_split = []

    # Iterate over each class
    for class_label in range(10):  # Fashion MNIST has 10 classes
        # Get indices of samples belonging to the current class
        class_indices = np.where(y_train == class_label)[0]

        # Shuffle indices
        np.random.shuffle(class_indices)

        # Select first N indices for training set
        train_indices = class_indices[:N]
        X_train_split.extend(X_train[train_indices])
        y_train_split.extend(y_train[train_indices])

        # Select remaining samples for testing set
        test_indices = class_indices[N:7000]  # 7000 - N
        X_test_split.extend(X_train[test_indices])
        y_test_split.extend(y_train[test_indices])

    # Convert the lists to numpy arrays
    X_train_split = np.array(X_train_split)
    X_test_split = np.array(X_test_split)
    y_train_split = np.array(y_train_split)
    y_test_split = np.array(y_test_split)

    # Flatten the images
    X_train_flat = X_train_split.reshape(-1, 784)
    X_test_flat = X_test_split.reshape(-1, 784)

    # Train the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_flat, y_train_split)

    # Make predictions on the test set
    y_pred = knn.predict(X_test_flat)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_split, y_pred)
    accuracies_N.append(accuracy)

    print(f"N = {N}, Accuracy = {accuracy:.4f}")


N = 3000, Accuracy = 0.8367
N = 4000, Accuracy = 0.8407
N = 5000, Accuracy = 0.8478


In the code above, I use 3000, 4000 and 5000 for N and we can seee that with 3000 the accuracy was 83.59%, with 4000 the accuracy was 84.37% and with 5000 the accuracy was 84.45%.

Based on these results, we can see as the size of the training set increased from 3000 to 5000, the accuracy of the classification improves, which means, having more training data tends to lead to better classification performance.

We can also notice the increase in accuracy from 4000 to 5000 is smaller compared to the increase from 3000 to 4000. This suggests that there might be diminishing returns in terms of accuracy improvement as we further increase the training set size.

To choose an optimal size for the training set, we can take the cross validation approach. We split the dataset into multiple training and testing sets, train the model on each training set, and evaluate its performance on the corresponding testing set. Repeat this process for different sizes of the training set and monitor the model's performance metrics  for each size. This This approach will make sure that the chosen size generalizes well to unseen data and helps in avoiding overfitting or underfitting.


The notion of optimality in this context can be defined as achieving the highest possible accuracy on unseen data while considering computational constraints and avoiding overfitting. By using cross-validation to assess performance across different training set sizes, we can select the size that strikes the best balance between model performance and generalization.

# Problem 3a:


---


New algorithm:
- Instead of selecting the first N1 samples for the training set, let's try tto create an evenly distributed training set.
- Then we will make sure that each class is repreesented proportionally in the training set. I feel like this will help prevent biad towards certain classes.
- After that we within each class, wee will randomly select N1 samples for the training set.

In [9]:
from sklearn.model_selection import train_test_split

# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Define N1
N1 = 4000  # or any other value between 1000 and 6000

# Initialize lists to store training and testing data
X_train_split = []
X_test_split = []
y_train_split = []
y_test_split = []

# Iterate over each class
for class_label in range(10):  # Fashion MNIST has 10 classes
    # Get indices of samples belonging to the current class
    class_indices = np.where(y_train == class_label)[0]

    # Randomly select N1 indices for the training set using stratified sampling
    train_indices, _ = train_test_split(class_indices, train_size=N1, stratify=y_train[class_indices])

    # Append selected training samples to the training data list
    X_train_split.extend(X_train[train_indices])
    y_train_split.extend(y_train[train_indices])

# Convert lists to numpy arrays
X_train_split = np.array(X_train_split)
y_train_split = np.array(y_train_split)

# Testing set is automatically formed from the remaining data
X_test_split = np.delete(X_train, train_indices, axis=0)
y_test_split = np.delete(y_train, train_indices)

# Print shapes of resulting arrays
print("Shape of X_train_split:", X_train_split.shape)
print("Shape of X_test_split:", X_test_split.shape)
print("Shape of y_train_split:", y_train_split.shape)
print("Shape of y_test_split:", y_test_split.shape)


Shape of X_train_split: (40000, 28, 28)
Shape of X_test_split: (56000, 28, 28)
Shape of y_train_split: (40000,)
Shape of y_test_split: (56000,)


# Problem 3b:


---

In [10]:
# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Define N
N = 4000  # or any other value between 1000 and 6000

# Initialize lists to store training and testing data for each class
X_train_split = []
X_test_split = []
y_train_split = []
y_test_split = []

# Iterate over each class
for class_label in range(10):  # Fashion MNIST has 10 classes
    # Get indices of samples belonging to the current class
    class_indices = np.where(y_train == class_label)[0]

    # Randomly select N indices for the training set using stratified sampling
    train_indices, _ = train_test_split(class_indices, train_size=N, stratify=y_train[class_indices])

    # Append selected training samples to the training data list
    X_train_split.extend(X_train[train_indices])
    y_train_split.extend(y_train[train_indices])

    test_indices = np.setdiff1d(class_indices, train_indices)
    X_test_split.extend(X_train[test_indices])
    y_test_split.extend(y_train[test_indices])

# Convert lists to numpy arrays
X_train_split = np.array(X_train_split)
X_test_split = np.array(X_test_split)
y_train_split = np.array(y_train_split)
y_test_split = np.array(y_test_split)

# Print shapes of resulting arrays
print("Shape of X_train_split:", X_train_split.shape)
print("Shape of X_test_split:", X_test_split.shape)
print("Shape of y_train_split:", y_train_split.shape)
print("Shape of y_test_split:", y_test_split.shape)


Shape of X_train_split: (40000, 28, 28)
Shape of X_test_split: (20000, 28, 28)
Shape of y_train_split: (40000,)
Shape of y_test_split: (20000,)


# Problem 3c

---

In [11]:
# Create KNN classifier with k = 20
knn_classifier = KNeighborsClassifier(n_neighbors=20)

# Flatten the images for KNN classifier
X_train_flatten = X_train_split.reshape(len(X_train_split), -1)
X_test_flatten = X_test_split.reshape(len(X_test_split), -1)

# Train the KNN classifier
knn_classifier.fit(X_train_flatten, y_train_split)

# Predict labels for the test set
y_pred = knn_classifier.predict(X_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.841


# Problem 3d

---

With this new algorithm we are getting accuracy of 0.841, it appears that the new split method, where each class has a training set consisting of N examples and a testing set consisting of 7000 - N examples, has yielded a reasonably high accuracy for the KNN classification task.